In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import seaborn as sns
import pandas as pd
import os

torch.set_default_device("cuda")

# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [ ]:
logits = torch.tensor([2.5, 2, 3])
probs = torch.nn.functional.softmax(logits, dim=-1)
print(probs)
ax = sns.barplot(probs.cpu())
ax.grid(axis="y")

In [ ]:
samples = torch.multinomial(probs, num_samples=1000, replacement=True)
print(torch.bincount(samples) / len(samples))
ax = sns.histplot(samples.cpu(), stat="probability")
ax.grid(axis="y")

In [ ]:
spec_logits = torch.tensor([3.0, 2.0, 1.0])
spec_probs = torch.nn.functional.softmax(spec_logits, dim=-1)
spec_samples = torch.multinomial(spec_probs, num_samples=1000, replacement=True)
print(spec_probs)

In [ ]:
ax = sns.histplot(spec_samples.cpu(), stat="probability")
ax.grid(axis="y")

In [ ]:
def rejection_sample(probs: torch.Tensor, spec_probs: torch.Tensor, idxs: torch.Tensor):
    """
    Target model distribution: q(x)
    Draft model distribution: p(x)
    Vectorized implementation
    """
    qs = probs[idxs]
    ps = spec_probs[idxs]
    rs = torch.rand(len(idxs))
    keep_mask = rs < torch.clamp(qs / ps, max=1.0)
    new_p = torch.clamp(probs - spec_probs, min=0.0)  # pseudo-probability
    new_samples = torch.multinomial(new_p, num_samples=len(idxs), replacement=True)
    return torch.where(keep_mask, idxs, new_samples)

In [ ]:
# rej_samples = torch.tensor([rejection_sample(probs, spec_probs, x) for x in spec_samples])
rej_samples = rejection_sample(probs, spec_probs, spec_samples)
ax = sns.histplot(rej_samples.cpu(), stat="probability")
ax.grid(axis="y")